我们这里给四缺陷打上伪标签

In [ ]:
import sys
sys.path.append('..')

In [ ]:
from py_file.M_attri import Att
from py_file.Get_Data import DATA
from py_file.data_set import MyDataSet
from py_file.func_Test import Test_Func
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch

In [ ]:
trans = transforms.Resize(224)  # ResNet模型适合的图片大小为224x244
# 输入的张量需要带着批次维度和通道维度

# 加载数据和模型

## 获取数据

In [ ]:
train_data_path = '/mnt/workspace/DATA/train_WM.npz'
train_data = np.load(train_data_path)
print(train_data.files)

In [ ]:
ori_wm = train_data['original_wm']
wm = train_data['denoise_wm']
label = train_data['label_name']
label_oh = train_data['label_one_hot']
four_ori_wm = []
four_wm = []
four_label = []
four_label_oh = []
for i in range(len(label_oh)):
    if label_oh[i].sum() == 4:
        four_ori_wm.append(np.array(ori_wm[i]))
        four_wm.append(np.array(wm[i]))  # 元素是tensor的列表无法转换为张量，所以这里把列表里的元素转换为np.array
        four_label.append(label[i])
        four_label_oh.append(label_oh[i])

In [ ]:
four_wm_tensor = trans(torch.tensor(four_wm, dtype=torch.float32))
four_wm_tensor = torch.reshape(four_wm_tensor,(-1,1,224,224))  # 给图片加上通道
print(four_wm_tensor.shape)

## 获取类别名称和多热编码对应的字典

In [ ]:
four_label_unique, label_indices = np.unique(four_label, return_index=True)
print(four_label_unique)
print(label_indices)
# print(two_label_unique[indices])
four_label_oh_unique ,label_oh_indices = np.unique(four_label_oh, axis=0, return_index=True)
print(four_label_oh_unique)
print(label_oh_indices)

In [ ]:
oh_dict = {}
for index in label_indices:
    oh_dict[four_label[index]] = four_label_oh[index]
print(oh_dict)

## 加载模型和更新过的语义

In [ ]:
import pickle
model = torch.load('model_saved_pseudo/train_single_two_three.pth')
with open('updated_semantic_1_2_3_4/updated_single_dict.pkl', 'rb') as file:
    single_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3_4/updated_two_dict.pkl', 'rb') as file:
    two_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3_4/updated_three_dict.pkl', 'rb') as file:
    three_defect_att = pickle.load(file)
with open('updated_semantic_1_2_3_4/updated_four_dict.pkl', 'rb') as file:
    four_defect_att = pickle.load(file)
model.eval()

In [ ]:
four_att = []
for l in four_label:
    four_att.append(four_defect_att[l])
four_att_tensor = torch.tensor(np.array(four_att), dtype=torch.float32)
print(four_att_tensor.shape)

In [ ]:
four_dataset = MyDataSet(four_wm_tensor, four_att_tensor)
four_loader = DataLoader(four_dataset, batch_size=32, shuffle=False)

# 定义预测标签的函数

In [ ]:
from py_file.func_Test import Test_Func
# 需要的函数都已经集成在了Test_Func里
func = Test_Func()

# 开始打伪标签

In [ ]:
real_label, predict_label = func.get_label(model, four_loader, four_defect_att, 'cos')

In [ ]:
real_label == four_label

In [ ]:
four_label = np.array(four_label)
predict_label = np.array(predict_label)
print(four_label.shape, predict_label.shape)

In [ ]:
predict_four_label_oh = []
for l in predict_label:
    predict_four_label_oh.append(oh_dict[l])
    
predict_four_label_oh = np.array(predict_four_label_oh)
print(predict_four_label_oh.shape)

# 保存打了伪标签的四故障样本

In [ ]:
four_ori_wm = np.array(four_ori_wm)
four_wm = np.array(four_wm)
four_label_oh = np.array(four_label_oh)
print(four_ori_wm.shape)
print(four_wm.shape)
print(four_label_oh.shape)

In [ ]:
np.savez('data_fake_label/four_fake_label_WM.npz', original_wm = four_ori_wm, label_one_hot = predict_four_label_oh,
 denoise_wm = four_wm, label_name = predict_label)